In [75]:
import numpy as np

def prod_nonzero_diag_v1(X):
    prod = 1
    for i in range(min(X.shape)):
        if X[i, i] != 0:
            prod *= X[i, i]
    return prod


In [76]:
def prod_nonzero_diag_v2(X):
    diag = np.diag(X)
    nonzero_diag = diag[diag != 0]
    return np.prod(nonzero_diag)


In [77]:
def prod_nonzero_diag_v3(X):
    return np.prod([X[i, i] for i in range(min(X.shape)) if X[i, i] != 0])


In [78]:
matrix = np.array([[1, 0, 3], [4, 0, 6], [7, 8, 9]])

%timeit prod_nonzero_diag_v1(matrix)
%timeit prod_nonzero_diag_v2(matrix)
%timeit prod_nonzero_diag_v3(matrix)


1.27 µs ± 7.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
9.27 µs ± 692 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
7.47 µs ± 175 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [79]:
def check_multiset_v1(x, y):
    return sorted(x) == sorted(y)


In [80]:
def check_multiset_v2(x, y):
    return np.array_equal(np.sort(x), np.sort(y))


In [81]:
def check_multiset_v3(x, y):
    ux, cx = np.unique(x, return_counts=True)
    uy, cy = np.unique(y, return_counts=True)
    return np.array_equal(ux, uy) and np.array_equal(cx, cy)


In [82]:
x = np.array([1, 2, 2, 3])
y = np.array([2, 3, 1, 2])

%timeit check_multiset_v1(x, y)
%timeit check_multiset_v2(x, y)
%timeit check_multiset_v3(x, y)


1.79 µs ± 44.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
7.02 µs ± 302 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
46.3 µs ± 3.99 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [83]:
def max_after_zero_v1(x):
    max_elem = float('-inf')
    for i in range(1, len(x)):
        if x[i-1] == 0 and x[i] > max_elem:
            max_elem = x[i]
    return max_elem
    

In [84]:
def max_after_zero_v2(x):
    zero_indices = np.where(x[:-1] == 0)[0] + 1
    if len(zero_indices) == 0:
        return None
    return np.max(x[zero_indices])


In [85]:
def max_after_zero_v3(x):
    return max([x[i] for i in range(1, len(x)) if x[i-1] == 0], default=None)


In [86]:
x = np.array([0, 1, 2, 0, 4, 5, 0, 7, 8, 0, 10])

%timeit max_after_zero_v1(x)
%timeit max_after_zero_v2(x)
%timeit max_after_zero_v3(x)


3.85 µs ± 77.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
8.54 µs ± 596 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
2.89 µs ± 283 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [87]:
def rgb_to_grayscale_v1(image):
    weights = np.array([0.299, 0.587, 0.114])
    height, width, _ = image.shape
    grayscale = np.empty((height, width))
    for i in range(height):
        for j in range(width):
            grayscale[i, j] = np.dot(image[i, j, :], weights)
    return grayscale


In [88]:
def rgb_to_grayscale_v2(image):
    weights = np.array([0.299, 0.587, 0.114])
    return np.dot(image[...,:3], weights)


In [89]:
def rgb_to_grayscale_v3(image):
    weights = np.array([0.299, 0.587, 0.114])
    return np.sum(image * weights, axis=2)


In [98]:
import numpy as np
import imageio.v2 as imageio

image_path = 'img.png'
image = imageio.imread(image_path)

if image.dtype != np.uint8:
    image = (image / np.max(image) * 255).astype(np.uint8)


In [99]:
import numpy as np

# image = np.random.randint(0, 256, (100, 100, 3), dtype=np.uint8)

%timeit rgb_to_grayscale_v1(image)
%timeit rgb_to_grayscale_v2(image)
%timeit rgb_to_grayscale_v3(image)


2.69 s ± 29.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
53.6 ms ± 5.43 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
43.5 ms ± 4.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [91]:
def rle_v1(x):
    values = []
    lengths = []
    count = 1
    for i in range(1, len(x)):
        if x[i] == x[i - 1]:
            count += 1
        else:
            values.append(x[i - 1])
            lengths.append(count)
            count = 1
    values.append(x[-1])
    lengths.append(count)
    return np.array(values), np.array(lengths)


In [92]:
def rle_v2(x):
    n = len(x)
    ends = np.nonzero(x[:-1] != x[1:])[0] + 1
    ends = np.r_[0, ends, n]
    values = x[ends[:-1]]
    lengths = np.diff(ends)
    return values, lengths


In [93]:
def rle_v3(x):
    values, lengths = [], []
    start = 0
    for end in range(1, len(x)):
        if x[end] != x[start]:
            values.append(x[start])
            lengths.append(end - start)
            start = end
    values.append(x[start])
    lengths.append(len(x) - start)
    return np.array(values), np.array(lengths)


In [94]:
x = np.array([1, 2, 2, 3, 3, 1, 1, 1])

%timeit rle_v1(x)
%timeit rle_v2(x)
%timeit rle_v3(x)


3.71 µs ± 82.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
26 µs ± 227 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
3.45 µs ± 18.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
